In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import mimetypes
import random
import cv2
from google.colab import drive

In [2]:
# Step 1: Extract the image data
drive.mount('/content/drive')

# Cargar los modelos de IA previamente entrenados
modelo_partes_cuerpo = tf.keras.models.load_model('/content/body_part_model.h5')
modelo_celulas_pulmon = tf.keras.models.load_model('/content/lung_model.h5')
modelo_celulas_colon = tf.keras.models.load_model('/content/colon_model.h5')
modelo_celulas_pecho = tf.keras.models.load_model('/content/breast_model.h5')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
#Mostrar imágenes visuales y su categoría
#Introducir el path de las imágenes

imagenes_path=['/content/12819_idx5_class0.png',
               '/content/12819_idx5_class1.png',
               '/content/16555_idx5_class0.png',
               '/content/16555_idx5_class1.png',
               '/content/8863_idx5_class0.png',
               '/content/8863_idx5_class1.png',
               '/content/8913_idx5_class0.png',
               '/content/8913_idx5_class1.png',
               '/content/10268_idx5_class0_aug_0.png',
               '/content/10268_idx5_class1_aug_0.png',
               '/content/colonca30.jpeg',
               '/content/colonca31.jpeg',
               '/content/colonca32.jpeg',
               '/content/colonn30.jpeg',
               '/content/colonn31.jpeg',
               '/content/colonn32.jpeg',
               '/content/lungaca30.jpeg',
               '/content/lungaca31.jpeg',
               '/content/lungaca32.jpeg',
               '/content/lungn30.jpeg',
               '/content/lungn31.jpeg',
               '/content/lungn32.jpeg',
               '/content/lungscc30.jpeg',
               '/content/lungscc31.jpeg',
               '/content/lungscc32.jpeg'
              ]


In [14]:
def preprocesado_imagen(imagen):
    imagen = cv2.imread(imagen)
    imagen_redimensionada = cv2.resize(imagen, (50, 50))
    imagen_rgb = cv2.cvtColor(imagen_redimensionada, cv2.COLOR_BGR2RGB)
    imagen_final=imagen_rgb/255 - 0.5
    return imagen_final


In [15]:
#Preprocesado
imagenes_preprocesadas=[]
for img in imagenes_path:
    imagenes_preprocesadas.append(preprocesado_imagen(img))

In [16]:
#Clasificador segun la parte del cuerpo

#Extraemos las predicciones
predicciones_cuerpo=modelo_partes_cuerpo.predict(np.array(imagenes_preprocesadas), verbose=0)
parte_cuerpo=[]

for i, celula in enumerate(predicciones_cuerpo):
    celula= np.argmax(celula)
    parte_cuerpo.append(celula)
    if (celula==0): celulas="células mamarias"
    elif (celula==1): celulas="células del colon"
    else: celulas= "células del pulmón"
    print(f'La imagen {i} corresponde a {celulas}')


La imagen 0 corresponde a células mamarias
La imagen 1 corresponde a células mamarias
La imagen 2 corresponde a células mamarias
La imagen 3 corresponde a células mamarias
La imagen 4 corresponde a células mamarias
La imagen 5 corresponde a células mamarias
La imagen 6 corresponde a células mamarias
La imagen 7 corresponde a células mamarias
La imagen 8 corresponde a células mamarias
La imagen 9 corresponde a células mamarias
La imagen 10 corresponde a células del colon
La imagen 11 corresponde a células del colon
La imagen 12 corresponde a células del colon
La imagen 13 corresponde a células del colon
La imagen 14 corresponde a células del colon
La imagen 15 corresponde a células del colon
La imagen 16 corresponde a células del pulmón
La imagen 17 corresponde a células del pulmón
La imagen 18 corresponde a células del pulmón
La imagen 19 corresponde a células del pulmón
La imagen 20 corresponde a células del pulmón
La imagen 21 corresponde a células del pulmón
La imagen 22 corresponde

In [17]:
#Función para devolver el tipo de célula
def tipo_celula_colon(tipo):
    threshold = 0.5
    tipo = np.where(tipo >= threshold, 1, 0)
    if (tipo==0): return "Adenocarcionoma (aca)"
    else: return "Células Sanas (n)"

#Función para devolver el tipo de célula
def tipo_celula_pecho(tipo):
    threshold = 0.5
    tipo = np.where(tipo >= threshold, 1, 0)
    if (tipo==0): return "Células Sanas (n)"
    else: return "Adenocarcionoma (aca)"

def tipo_celula_pulmon(tipo):
    tipo = np.argmax(tipo)
    if tipo == 0:
        return "Adenocarcionoma (aca)"
    elif tipo == 1:
        return "Células Sanas (n)"
    elif tipo == 2:
        return "Carcinoma de Celulas Escamosas de Pulmón (scc)"
    else:
        return "no identificado"


In [18]:
#Función para devolver la célula
def celula_cuerpo(tipo):
    if (tipo==0): return "mamario"
    elif (tipo==1): return "de colon"
    else: return "de pulmón"

In [19]:
#Modelos de predicción de células benignas y carcionomas
for i, img in enumerate(np.array(imagenes_preprocesadas)):
    cuerpo= celula_cuerpo(parte_cuerpo[i])
    if (cuerpo=='mamario'): #Pecho
        celula= tipo_celula_pecho(modelo_celulas_pecho.predict(np.expand_dims(img, axis=0), verbose=0))
    elif (cuerpo=='de colon'): #Colon
        celula= tipo_celula_colon(modelo_celulas_colon.predict(np.expand_dims(img, axis=0), verbose=0))
    elif (cuerpo=='de pulmón'): #pulmon
      celula= tipo_celula_pulmon(modelo_celulas_pulmon.predict(np.expand_dims(img, axis=0), verbose=0))
    else:
      continue
    print(f"La imagen {i} corresponde con tejido {cuerpo} que contiene {celula}")

La imagen 0 corresponde con tejido mamario que contiene Células Sanas (n)
La imagen 1 corresponde con tejido mamario que contiene Adenocarcionoma (aca)
La imagen 2 corresponde con tejido mamario que contiene Células Sanas (n)
La imagen 3 corresponde con tejido mamario que contiene Adenocarcionoma (aca)
La imagen 4 corresponde con tejido mamario que contiene Células Sanas (n)
La imagen 5 corresponde con tejido mamario que contiene Adenocarcionoma (aca)
La imagen 6 corresponde con tejido mamario que contiene Células Sanas (n)
La imagen 7 corresponde con tejido mamario que contiene Adenocarcionoma (aca)
La imagen 8 corresponde con tejido mamario que contiene Células Sanas (n)
La imagen 9 corresponde con tejido mamario que contiene Adenocarcionoma (aca)
La imagen 10 corresponde con tejido de colon que contiene Adenocarcionoma (aca)
La imagen 11 corresponde con tejido de colon que contiene Adenocarcionoma (aca)
La imagen 12 corresponde con tejido de colon que contiene Adenocarcionoma (aca)
